# Q# Interop with OpenQASM

The modern QDK provides interoperability with OpenQASM 3 programs built upon the core Q# compiler infrastructure.

This core enables integration and local resource estimation without relying on external tools. Users are able to estimate resources for their OpenQASM programs locally (see the [resource estimation with OpenQASM sample notebook](../../estimation/estimation-openqasm.ipynb)), leveraging the Q# compiler's capabilities for analysis, transformation, code generation, and simulation. This also enables the generation of QIR from OpenQASM progams leveraging the [modern QDKs advanced code generation capabilities](https://devblogs.microsoft.com/qsharp/integrated-hybrid-support-in-the-azure-quantum-development-kit/).

This includes support for classical instructions available in OpenQASM such as for loops, if statements, switch statements, while loops, binary expresssions, and more.

### Simulating OpenQASM programs

In [ ]:
from qsharp.openqasm import run

source = """
include "stdgates.inc";
bit[2] c;
qubit[2] q;
h q[0];
cx q[0], q[1];
c = measure q;
"""

run(source, as_bitstring=True)

Import the Q# module.

This enables the `%%qsharp` magic and initializes a Q# interpreter singleton.

In [ ]:
import qsharp
qsharp.init(target_profile=qsharp.TargetProfile.Base)

### Run OpenQASM 3 Code in interactive session
Interactive sessions have different semantics from program execution. We no longer have inferred output and input. Instead we treat qasm lines as code fragments and interpret them one at a time (though they are all compiled together). Due to scoping rules in qasm3, all code used in the the program must be defined in the snippet and can't use compilation state from other cells or calls.

We can add an optional name parameter to compile the program into a callable operation in the interactive session.

In [ ]:
from qsharp.openqasm import import_openqasm

source = """
include "stdgates.inc";
bit[2] c;
qubit[2] q;
h q[0];
cx q[0], q[1];
c = measure q;
"""

import_openqasm(source, name="bell")

With the OpenQASM program loaded into a callable name `bell`, we can now import it via the QDK's Python bindings:

In [ ]:
from qsharp.code import bell
bell()

Additionally, since it is defined in the session, we can run it directly from a Q# cell:

In [ ]:
%%qsharp
bell()

This also unlocks all of the other `qsharp` package functionality. Like noisy simulation:

In [ ]:
from qsharp_widgets import Histogram

Histogram(qsharp.run("bell()", shots=1000, noise=qsharp.DepolarizingNoise(0.01)))

Circuit rendering:

In [ ]:
qsharp.circuit(qsharp.code.bell)

Circuit widget rendering:

In [ ]:
from qsharp_widgets import Circuit
Circuit(qsharp.circuit(qsharp.code.bell))

Code generation:

In [ ]:
print(qsharp.compile(bell))

We can also define input for the compiled OpenQASM code so that we can parameterize input:

In [ ]:
from qsharp.openqasm import import_openqasm

source = """
include "stdgates.inc";
input float theta;
bit[2] c;
qubit[2] q;
rx(theta) q[0];
rx(-theta) q[1];
c = measure q;
"""

import_openqasm(source, name="parameterized_program")


In [ ]:
from qsharp.code import parameterized_program
print(qsharp.compile(parameterized_program, 1.57))